# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/Users/admin/Desktop/Data_engineer_Udacity/Data_Modeling_NoSQL/home-2


In [3]:
!pwd

/Users/admin/Desktop/Data_engineer_Udacity/Data_Modeling_NoSQL/home-2


In [4]:
!ls -a

.                                  event_data
..                                 event_datafile_new.csv
.ipynb_checkpoints                 images
Project_1B_ Project_Template.ipynb


In [5]:
!ls ./event_data -a

ls: -a: No such file or directory
./event_data:
2018-11-01-events.csv 2018-11-11-events.csv 2018-11-21-events.csv
2018-11-02-events.csv 2018-11-12-events.csv 2018-11-22-events.csv
2018-11-03-events.csv 2018-11-13-events.csv 2018-11-23-events.csv
2018-11-04-events.csv 2018-11-14-events.csv 2018-11-24-events.csv
2018-11-05-events.csv 2018-11-15-events.csv 2018-11-25-events.csv
2018-11-06-events.csv 2018-11-16-events.csv 2018-11-26-events.csv
2018-11-07-events.csv 2018-11-17-events.csv 2018-11-27-events.csv
2018-11-08-events.csv 2018-11-18-events.csv 2018-11-28-events.csv
2018-11-09-events.csv 2018-11-19-events.csv 2018-11-29-events.csv
2018-11-10-events.csv 2018-11-20-events.csv 2018-11-30-events.csv


In [6]:
!rm -r ./event_data/.ipynb_checkpoints

rm: ./event_data/.ipynb_checkpoints: No such file or directory


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [7]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)

 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [8]:
print(len(full_data_rows_list))

8056


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [10]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [11]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [12]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [13]:
# Here we want to distribute our data across nodes via sessionId and
# itemInSession is used as a clustering column that helps making PK unique and sorts the data
query = "CREATE TABLE IF NOT EXISTS artist_song "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float,\
                                                    PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
# we getting artist name, song title and song length corresponding to sessionId 338 and itemInSession 4
query = "select artist, song, length from artist_song WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [16]:
# distributing data based on the session and user (sessions belong to the same user, hence we need them on the same machine)
# itemInSession is used as clustering column for PK uniqueness
query = "CREATE TABLE IF NOT EXISTS user_artist_song "
query = query + "(sessionId int, userId int, itemInSession int, artist text, song text, firstName text,\
                                        lastName text, PRIMARY KEY ((sessionId, userId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_artist_song (sessionId, userId, itemInSession, artist, song,\
                                                                             firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0],\
                                                                    line[9], line[1], line[4])) 

In [18]:
# we want info about song and artist that particular user was listening on a particular session
query = "select artist, song, firstName, lastName \
                                    from user_artist_song WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [19]:
# we want every user that listened specific song, hence
# we partition by song title and add artistId for uniqueness again
query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + "(song text, userId int, firstName text, lastName text,\
                                         PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))               

In [21]:
# getting full name of all the users that have listened the specific song
query = "select firstName, lastName from user_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [22]:
query = "drop table artist_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_artist_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()